<a href="https://colab.research.google.com/github/Sathyaseelan-Geo-Spatial-Lab/Climate-Change-Lab/blob/main/EnMAP_CH4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Try this first:
# !pip install git+https://github.com/spaceml-org/STARCOP.git

# But for Google Colab (as of January 2025) instead use:
!pip install git+https://github.com/spaceml-org/georeader.git
!pip install torch==2.0.0 torchvision==0.15.1 torchtext==0.15.1 pytorch-lightning==2.2
!pip install fsspec gcsfs omegaconf kornia==0.6.7  torchmetrics==0.10.0 wandb segmentation_models_pytorch hydra-core ipython rasterio  geopandas ipykernel matplotlib scikit-image scikit-learn wandb==0.13.3
!pip install netCDF4 spectral

!pip install huggingface_hub[cli,torch]
!pip install matplotlib-scalebar

In [ ]:
!git clone https://github.com/spaceml-org/STARCOP.git

In [ ]:
%cd STARCOP

In [ ]:
from huggingface_hub import hf_hub_download
from georeader.readers import emit
from starcop.models import mag1c_emit
from georeader import plot
import starcop
from starcop.models.model_module import ModelModule
import os
import torch
import omegaconf
import numpy as np
import matplotlib.pyplot as plt
from starcop.models.utils import padding
import georeader

link = emit.get_radiance_link("EMIT_L1B_RAD_001_20220827T060753_2223904_013.nc")
print("Download", link)

# NASA's data archive requires creating an account for downloading EMIT files directly.
# Add the user and password of the NASA Earthdata portal (https://search.earthdata.nasa.gov/search)
product = emit.download_product(link,
                                auth=("user","password"))

rst = emit.EMITImage(product)
rst

In [ ]:
# alernatively for the sake of this demo, we also uploaded this scene on our gdrive (but if these two files are missing, please use the above download instead)
!gdown 1jyFejO80Q82qUZ5tRPaDTqsBHefuXYa0
!gdown 1a9k2YZDXfa4g95KlqCcKfkYcaA-QegYU
!ls

rst = emit.EMITImage("EMIT_L1B_RAD_001_20220827T060753_2223904_013.nc")
rst

In [ ]:
wavelengths_read = np.array([2000, 550, 460])

bands_read = np.argmin(np.abs(wavelengths_read[:, np.newaxis] - rst.wavelengths), axis=1).tolist()
rst_rgb = rst.read_from_bands(bands_read)
rst_rgb

In [ ]:
rgb_raw = rst_rgb.load_raw(transpose=True)
plt.imshow((rgb_raw/12).clip(0,1).transpose(1,2,0))

In [ ]:
mfoutput, albedo = mag1c_emit.mag1c_emit(rst, column_step=2, georreferenced=False)
mfoutput

In [ ]:
plt.imshow(mfoutput, vmin=0,vmax=1750)
plt.title("$\Delta$CH$_4$ [ppm x m]")
plt.colorbar()

In [ ]:
from huggingface_hub import hf_hub_download
# experiment_name = "hyperstarcop_mag1c_only"
experiment_name = "hyperstarcop_mag1c_rgb"
subfolder_local = f"models/{experiment_name}"
config_file = hf_hub_download(repo_id="isp-uv-es/starcop",subfolder=subfolder_local, filename="config.yaml",
                              local_dir=".", local_dir_use_symlinks=False)
model_file = hf_hub_download(repo_id="isp-uv-es/starcop",subfolder=subfolder_local,
                             filename="final_checkpoint_model.ckpt",
                              local_dir=".", local_dir_use_symlinks=False)

In [ ]:
hsi_model_path = os.path.join(subfolder_local, "final_checkpoint_model.ckpt")
hsi_config_path =  os.path.join(subfolder_local, "config.yaml")

device = torch.device("cpu")
config_general = omegaconf.OmegaConf.load(os.path.join(os.path.dirname(os.path.abspath(starcop.__file__)), 'config.yaml'))

def load_model_with_emit(model_path, config_path):
    config_model = omegaconf.OmegaConf.load(config_path)
    config = omegaconf.OmegaConf.merge(config_general, config_model)

    model = ModelModule.load_from_checkpoint(model_path, settings=config)
    model.to(device)
    model.eval() # !

    print("Loaded model with",model.num_channels,"input channels")

    return model, config

hsi_model, hsi_config = load_model_with_emit(hsi_model_path, hsi_config_path)
print("successfully loaded HyperSTARCOP model!")

In [ ]:
# Data re-normalisation to fit the range of our models
# (these params were found from statistics of the training datasets and data ranges)

# DIV the EMIT data by
MAGIC_DIV_BY = 240.
RGB_DIV_BY = 20.
# clipping too large values
MAGIC_CLIP_TO = [0.,2.]
RGB_CLIP_TO =   [0.,2.]
# MULT_BY to get it back to the range we saw in the AVIRIS data ...
MAGIC_MULT_BY = 1750.
RGB_MULT_BY =   60.


# NORMALISE
# emit rgb has max ~22
e_mag1c = np.clip(mfoutput / MAGIC_DIV_BY, MAGIC_CLIP_TO[0], MAGIC_CLIP_TO[1]) * MAGIC_MULT_BY
e_rgb = np.clip(rgb_raw / RGB_DIV_BY, RGB_CLIP_TO[0], RGB_CLIP_TO[1]) * RGB_MULT_BY
input_data = np.concatenate([e_mag1c[None], e_rgb], axis=0)
input_data.shape

In [ ]:
crs_utm = georeader.get_utm_epsg(rst.footprint("EPSG:4326"))
emit_image_utm = rst.to_crs(crs_utm)

In [ ]:
mfgeo = emit_image_utm.georreference(mfoutput, fill_value_default=-1)
predgeo = emit_image_utm.georreference(pred[0], fill_value_default=0)
rgbgeo = emit_image_utm.georreference(rgb_raw, fill_value_default=-1)

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(18,6),sharey=True)

rgbgeomask = np.any(rgbgeo.values == -1, axis=0,keepdims=False)
rgbplot = (rgbgeo/12).clip(0,1)
rgbplot.values[:, rgbgeomask] = -1
plot.show(rgbplot, ax=ax[0], title= "RGB",mask=True, add_scalebar=True)
plot.show(mfgeo, ax=ax[1], title= "$\Delta$CH$_4$ [ppm x m]",mask=True,vmin=0, vmax=1750,
         add_colorbar_next_to=True, add_scalebar=True)
plot.show(predgeo, ax=ax[2], title= "pred", mask=True, vmin=0, vmax=1, add_scalebar=True,
          add_colorbar_next_to=True)